# Raw Images reduction 

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : 9 juin 2021
- update : 7 july 2021, July 22th 2021

- inspired from Oliver viewimg

- kernel : anaconda3_auxtel_v2

# Selection Flag

In [1]:
FLAG_SPRING_2021=False
FLAG_SUMMER_2021=True

In [2]:
from astropy.io import fits
import sys, platform, os
#from astropy.visualization import simple_norm
#from astropy.visualization import imshow_norm, MinMaxInterval, SqrtStretch
import astropy.visualization  as aspv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm

In [3]:
def get_image_data(rep,num):
    dir_path = os.path.dirname(os.path.realpath(rep))
    end=str(num)+'.fits'
    print(end)
    filename='NOT_FOUND'
    for root, dirs, files in os.walk(dir_path):
        for filen in files:
            if filen.endswith(end):
                filename= filen

    if filename=='NOT_FOUND':
        return -99
    print(filename)

    hdulist = fits.open(rep+filename)
    data=hdulist[0].data

    hdulist.close()
    return data



In [4]:
def sub_med_hovc(data,ol,oh,ll,lh):
    sz=np.shape(data)[0]
    wd=np.shape(data)[1]
    newdat = np.zeros((sz-(oh-ol+1)-(lh-ll+1),wd))

    ovl=np.median(data[ol:oh+1,:],axis=0)
    ovh=np.median(data[ll:lh+1,:],axis=0)
    newdat[0:ol,:]=data[0:ol,:]-ovl#[:,np.newaxis]
    newdat[ol:,:]=data[lh+1:,:]-ovh#[:,np.newaxis]
    return newdat


In [5]:
def remove_vovc(data,losz,rosz,nov):
    sz=np.shape(data)[0]
    wd=np.shape(data)[1]

    osz=losz+rosz
    newl=wd-nov*(osz) #-1
    newdat = np.zeros((sz,newl))

    print("sz="+str(sz)+" wd="+str(wd)+" newl="+str(newl))
    print(osz)
    width = newl/8
    print(width)

    indx=(np.arange(newl)).astype('int')

    indx = indx + osz*((indx/width).astype('int')+1)-losz
    newdat=data[:,indx]
    return newdat

In [6]:
#1724 1791
def viewimg_raw(rep,num):
    z=get_image_data(rep,num)
    print(np.shape(z))
    sz=np.shape(z)
    zc=sub_med_hovc(z,2000,2047,2048,2095)

    zd=remove_vovc(zc,3,64,8)

    #plt.figure()
    #plt.imshow(z,aspect='auto',interpolation='none',cmap='gist_ncar')
    #plt.colorbar()
    plt.figure()
    plt.imshow(zc,aspect='auto',interpolation='none',cmap='gist_ncar')
    plt.colorbar()
    plt.figure()
    plt.imshow(zd,aspect='auto',interpolation='none',cmap='gist_ncar')
    plt.colorbar()
    print(np.shape(zd))

In [7]:
def viewimg_cln(rep,num):
    filename=get_name(rep,num)
    plt.figure(figsize=(13,9.5))
    if filename=='NOT_FOUND':
        return "unknown file"

    hdl=fits.open(rep+filename)
    zd=hdl[1].data
    norm = aspv.simple_norm(zd,stretch='sqrt')
    plt.imshow(zd,origin='lower',aspect='auto',interpolation='none',cmap='gist_ncar',norm=norm,vmin=-100,vmax=3100)
    plt.colorbar()
    print(repr(hdl[0].header))

In [8]:
def get_name(rep,num):
    dir_path = (os.path.realpath(rep))
    end=str(num)+'.fits'
    print(end)
    print(dir_path)
    filename='NOT_FOUND'
    for root, dirs, files in os.walk(dir_path):
        for filen in files:
            if filen.endswith(end):
                filename= filen
    print(filename)
    return filename


In [9]:
# loop over subdirs, may be wrong
def cor_img(rep,num,repsave):

    dir_path = os.path.dirname(os.path.realpath(rep))
    end=str(num)+'.fits'
    print(end)
    filename='NOT_FOUND'
    for root, dirs, files in os.walk(dir_path):
        for filen in files:
            if filen.endswith(end):
                filename= filen

    if filename=='NOT_FOUND':
        return -99
    print(filename)

    hdulist = fits.open(rep+filename)
    hh=hdulist[0].header


    z=hdulist[0].data
    hdulist.close()
    sz=np.shape(z)
    zc=sub_med_hovc(z,2000,2047,2048,2095)

    zd=remove_vovc(zc,3,64,8)
    #sp=np.shape(zd)
    del hh['DATASEC']
    #hh['NAXIS1']=sp[1]
    #hh['NAXIS2']=sp[0]

    hh.set('HA', hh['HASTART'])

    print(repr(hh))
    print (" ---> ")
    hdui = fits.ImageHDU(np.flip(zd,axis=0))
    hdu = fits.PrimaryHDU()
    nhd=hdu.header
    
    for hk in hh[5:] :
        if hk != 'COMMENT':
            nhd.set(hk,hh[hk])
    print(repr(nhd))

    #   hdu.writeto(repsave+"Cor_"+filename,overwrite=True)
    new_hdul = fits.HDUList([hdu, hdui])
    new_hdul.writeto(repsave+"Cor_"+filename,overwrite=True)
    return 0


In [10]:
# does not loop over subdirs
def cor_img_v2(rep,num,repsave):

    #dir_path = os.path.dirname(os.path.realpath(rep))
    dir_path = rep
    end=str(num)+'.fits'
    
    print(end)
    filename='NOT_FOUND'
    for root, dirs, files in os.walk(dir_path):
        print("dir_path = ",dir_path)   
        print("root=",root)
        print("dirs=",dirs)
        
        for filen in files:
            if filen.endswith(end):
                filename= filen

    if filename=='NOT_FOUND':
        return -99
    print(filename)

    hdulist = fits.open(rep+filename)
    hh=hdulist[0].header


    z=hdulist[0].data
    print("z.shape=",z.shape)
    hdulist.close()
    
    
    sz=np.shape(z)
    zc=sub_med_hovc(z,2000,2047,2048,2095)
    zd=remove_vovc(zc,3,64,8)
    
    
    #sp=np.shape(zd)
    del hh['DATASEC']
    #hh['NAXIS1']=sp[1]
    #hh['NAXIS2']=sp[0]

    hh.set('HA', hh['HASTART'])

    #print(repr(hh))
    #print (" ---> ")
    
    
    hdui = fits.ImageHDU(np.flip(zd,axis=0))
    hdu = fits.PrimaryHDU()
    
    nhd=hdu.header
    
    for hk in hh[5:] :
        if hk != 'COMMENT':
            nhd.set(hk,hh[hk])
    print(repr(nhd))

    #   hdu.writeto(repsave+"Cor_"+filename,overwrite=True)
    new_hdul = fits.HDUList([hdu, hdui])
    new_hdul.writeto(repsave+"Cor_"+filename,overwrite=True)
    return 0


# Work

# Summer 2021

In [11]:
if FLAG_SUMMER_2021:
    #intputdir_day="/sps/lsst/groups/auxtel/data/2021/holo/raw/2021-07-06/"
    #outputdir_day="/sps/lsst/groups/auxtel/data/2021/holo/localreduced/2021-07-06/"
    
    intputdir_day="/sps/lsst/groups/auxtel/data/2021/holo/raw/2021-07-07/"
    outputdir_day="/sps/lsst/groups/auxtel/data/2021/holo/localreduced/2021-07-07/"
    

    list_of_files=os.listdir(intputdir_day)
    

In [12]:
! ls /sps/lsst/groups/auxtel/data/2021/micellanous/raw/2021-07-07

ls: cannot access /sps/lsst/groups/auxtel/data/2021/micellanous/raw/2021-07-07: No such file or directory


In [13]:
! ls /sps/lsst/groups/auxtel/data/2021/micellanous/localreduced/2021-07-07

ls: cannot access /sps/lsst/groups/auxtel/data/2021/micellanous/localreduced/2021-07-07: No such file or directory


In [14]:
list_of_files.sort()
list_of_files

['holo4_003_RG610_HD160617_20210707_000234.fits',
 'holo4_003_RG610_HD160617_20210707_000235.fits',
 'holo4_003_RG610_HD160617_20210707_000236.fits',
 'holo4_003_RG610_HD160617_20210707_000237.fits',
 'holo4_003_RG610_HD160617_20210707_000238.fits',
 'holo4_003_RG610_HD160617_20210707_000239.fits',
 'holo4_003_RG610_HD160617_20210707_000240.fits',
 'holo4_003_RG610_HD160617_20210707_000241.fits',
 'holo4_003_RG610_HD160617_20210707_000242.fits',
 'holo4_003_RG610_HD160617_20210707_000243.fits',
 'holo4_003_RG610_HD160617_20210707_000244.fits',
 'holo4_003_RG610_HD160617_20210707_000245.fits',
 'holo4_003_RG610_HD160617_20210707_000246.fits',
 'holo4_003_RG610_HD160617_20210707_000247.fits',
 'holo4_003_RG610_HD160617_20210707_000306.fits',
 'holo4_003_RG610_HD160617_20210707_000307.fits',
 'holo4_003_RG610_HD160617_20210707_000308.fits',
 'holo4_003_empty_HD160617_20210707_000248.fits',
 'holo4_003_empty_HD160617_20210707_000249.fits',
 'holo4_003_empty_HD160617_20210707_000250.fits',


In [15]:
all_num = []

for filename in list_of_files:
    num=filename.split("_")[-1].split(".")[0]
    all_num.append(num)
    
all_num=np.array(all_num)

In [16]:
all_num.sort()

In [17]:
all_num

array(['000234', '000235', '000236', '000237', '000238', '000239',
       '000240', '000241', '000242', '000243', '000244', '000245',
       '000246', '000247', '000248', '000249', '000250', '000251',
       '000252', '000253', '000254', '000255', '000256', '000257',
       '000258', '000259', '000260', '000261', '000262', '000263',
       '000264', '000265', '000266', '000267', '000268', '000269',
       '000270', '000271', '000272', '000273', '000274', '000275',
       '000276', '000277', '000278', '000279', '000280', '000281',
       '000282', '000283', '000284', '000285', '000286', '000287',
       '000288', '000289', '000290', '000291', '000292', '000293',
       '000294', '000295', '000296', '000297', '000298', '000299',
       '000300', '000301', '000302', '000303', '000304', '000305',
       '000306', '000307', '000308', '000309', '000317', '000318',
       '000319', '000320', '000321', '000322', '000323', '000324',
       '000325', '000326', '000327', '000328', '000329', '0003

In [18]:
len(all_num)

124

In [19]:
for num in all_num:
    #cor_img(intputdir_day_1,num,outputdir_day_1)
    
    print("================================", num,"=========================================")
    cor_img_v2(intputdir_day,num,outputdir_day)

================================ 000234 =========================================
000234.fits
dir_path =  /sps/lsst/groups/auxtel/data/2021/holo/raw/2021-07-07/
root= /sps/lsst/groups/auxtel/data/2021/holo/raw/2021-07-07/
dirs= []
holo4_003_RG610_HD160617_20210707_000234.fits
z.shape= (4096, 4608)
sz=4000 wd=4608 newl=4072
67
509.0


SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
ORIGIN  = 'LSST DM Header Service'                                              
TIMESYS = 'TAI     '                                                            
DATE    = '2021-07-08T02:50:02.441'                                             
DATE-OBS= '2021-07-08T02:50:31.569'                                             
DATE-BEG= '2021-07-08T02:50:31.569'                                             
DATE-END= '2021-07-08T02:50:36.812'                                             
MJD     =     59403.1180838135                                                  
MJD-OBS =     59403.1184209388                                                  
MJD-BEG =     59403.11842093

In [20]:
plt.imshow(z,origin="lower",norm=LogNorm())

NameError: name 'z' is not defined

In [ ]:
sz=np.shape(z)
zc=sub_med_hovc(z,2000,2047,2048,2095)
zd=remove_vovc(zc,3,64,8)

In [ ]:
plt.imshow(zc,origin="lower",norm=LogNorm())

In [ ]:
plt.imshow(zd,origin="lower",norm=LogNorm())